In [1]:
import openmatrix as omx
import numpy as np
import scipy.io
import pandas as pd

# Load the OMX file using openmatrix
with omx.open_file(r"data\TlfOutputFromModel\tmp_TLF Idx - Dist.omx", 'r') as f:
    matrix_names = f.list_matrices()
    print("Available matrices:", matrix_names)
    # Use the first matrix name, or replace with the correct one after inspecting the list
    matrix_hbsch = f['Idx_HBSch_Sc'][:]

print(matrix_hbsch)

Available matrices: ['Idx_HBOth', 'Idx_HBSch_Pr', 'Idx_HBSch_Sc', 'Idx_HBShp', 'Idx_HBW', 'Idx_HV', 'Idx_IX', 'Idx_IX_HV', 'Idx_IX_LT', 'Idx_IX_MD', 'Idx_LT', 'Idx_MD', 'Idx_NHBNW', 'Idx_NHBW', 'Idx_XI', 'Idx_XI_HV', 'Idx_XI_LT', 'Idx_XI_MD']
[[  1.   1.  13. ... 151. 132. 132.]
 [  1.   1.  12. ... 151. 131. 132.]
 [ 13.  12.   1. ... 148. 128. 129.]
 ...
 [151. 151. 148. ...   1.  26.  25.]
 [132. 132. 129. ...  26.   1.   3.]
 [132. 132. 129. ...  25.   3.   1.]]


In [2]:
input_df = pd.DataFrame([
    ['data/TlfOutputFromModel/obs_pa_table_2023.omx', 'HBSCH_PR', 2023, 'Primary'],
    ['data/TlfOutputFromModel/obs_pa_table_2023.omx', 'HBSCH_SC', 2023, 'Secondary'],
    ['data/TlfOutputFromModel/obs_pa_table_2012.omx', 'HBSCH_PR', 2012, 'Primary'],
    ['data/TlfOutputFromModel/obs_pa_table_2012.omx', 'HBSCH_SC', 2012, 'Secondary']
], columns=['omx_path', 'matrix_name', 'year', 'type'])

import numpy as np
import pandas as pd
import openmatrix as omx

# input_df already provided
all_long_frames = []

# Open each OMX file once, process all requested matrices inside it
for omx_path, sub in input_df.groupby('omx_path', sort=False):
    with omx.open_file(omx_path, 'r') as f:
        available = set(f.list_matrices())
        for _, row in sub.iterrows():
            name = row['matrix_name']
            if name not in available:
                print(f"WARNING: '{name}' not found in {omx_path}. "
                      f"Available: {sorted(available)}")
                continue

            # Read matrix and convert to sparse long (non-zero only)
            M = f[name][:]  # numpy array
            nz_rows, nz_cols = np.nonzero(M)

            if nz_rows.size == 0:
                # Nothing to add for this matrix
                continue

            df_long_nz = pd.DataFrame({
                "pTAZID": (nz_rows + 1).astype(np.int64),  # 1-based IDs
                "aTAZID": (nz_cols + 1).astype(np.int64),
                "value":  M[nz_rows, nz_cols]
            })

            # Add requested columns
            df_long_nz["year"] = int(row["year"])
            df_long_nz["type"] = row["type"]
            df_long_nz["matrix_name"] = name

            all_long_frames.append(df_long_nz)

# Concatenate all pieces
if all_long_frames:
    df_all = pd.concat(all_long_frames, ignore_index=True)
else:
    df_all = pd.DataFrame(columns=["pTAZID", "aTAZID", "value", "year", "type"])

display(df_all)


,pTAZID,aTAZID,value,year,type,matrix_name
0,6,10,54.86,2023,Primary,HBSCH_PR
1,6,19,344.96,2023,Primary,HBSCH_PR
2,12,15,179.30,2023,Primary,HBSCH_PR
3,15,13,145.85,2023,Primary,HBSCH_PR
4,23,23,41.87,2023,Primary,HBSCH_PR
...,...,...,...,...,...,...
1822,2548,2548,395.29,2012,Secondary,HBSCH_SC
1823,2555,2555,339.18,2012,Secondary,HBSCH_SC
1824,2557,2525,282.90,2012,Secondary,HBSCH_SC
1825,2560,2560,90.10,2012,Secondary,HBSCH_SC


In [3]:
# get row and column id for any matrix value==1
ltmile_row_ids, ltmile_col_ids = np.where(matrix_hbsch == 1)  # this is actually bin <1 mile

# create dataframe and add values for each as new columns
df_ltmile = pd.DataFrame({
    'pTAZID': ltmile_row_ids + 1,
    'aTAZID': ltmile_col_ids + 1
})

df_ltmile


,pTAZID,aTAZID
0,1,1
1,1,2
2,1,3605
3,2,1
4,2,2
...,...,...
21988,3625,3625
21989,3626,3626
21990,3627,3627
21991,3628,3628


In [4]:
hbsch_sc_ltmile = pd.merge(df_ltmile, df_long_nz, on=['pTAZID', 'aTAZID'])
hbsch_sc_ltmile


,pTAZID,aTAZID,value,year,type,matrix_name
0,16,16,99.46,2012,Secondary,HBSCH_SC
1,24,24,92.04,2012,Secondary,HBSCH_SC
2,25,25,37.70,2012,Secondary,HBSCH_SC
3,44,35,39.06,2012,Secondary,HBSCH_SC
4,45,35,28.99,2012,Secondary,HBSCH_SC
...,...,...,...,...,...,...
213,2526,2524,75.48,2012,Secondary,HBSCH_SC
214,2530,2530,73.68,2012,Secondary,HBSCH_SC
215,2548,2548,395.29,2012,Secondary,HBSCH_SC
216,2555,2555,339.18,2012,Secondary,HBSCH_SC


In [5]:
hbsch_sc_sametaz = hbsch_sc_ltmile[hbsch_sc_ltmile['pTAZID'] == hbsch_sc_ltmile['aTAZID']]
hbsch_sc_sametaz


,pTAZID,aTAZID,value,year,type,matrix_name
0,16,16,99.46,2012,Secondary,HBSCH_SC
1,24,24,92.04,2012,Secondary,HBSCH_SC
2,25,25,37.70,2012,Secondary,HBSCH_SC
5,45,45,151.88,2012,Secondary,HBSCH_SC
6,48,48,39.66,2012,Secondary,HBSCH_SC
...,...,...,...,...,...,...
211,2468,2468,144.96,2012,Secondary,HBSCH_SC
214,2530,2530,73.68,2012,Secondary,HBSCH_SC
215,2548,2548,395.29,2012,Secondary,HBSCH_SC
216,2555,2555,339.18,2012,Secondary,HBSCH_SC


In [6]:
display(hbsch_sc_sametaz['value'].sum())
display(hbsch_sc_ltmile['value'].sum())
display(df_long_nz['value'].sum())

np.float64(44000.45999999999)

np.float64(48806.33)

np.float64(99435.46)

# Find records in HTS that match TAZIDs

# Map HTS HBSch_PR and SC Records

In [7]:
# import global TDM functions
import sys
sys.path.insert(0, '../Resources/2-Python/global-functions')
import BigQuery
import geopandas as gpd

client = BigQuery.getBigQueryClient_Confidential2023UtahHTS()


bhereth
C:\Users\bhereth\confidential-2023-utah-hts-db5335615978.json


In [8]:
bq_taz_ustm3 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.geometries.ustm_v3_taz_2021_09_22_geog').to_dataframe()
gdf_taz = gpd.GeoDataFrame(bq_taz_ustm3)
gdf_taz_wf = gdf_taz[gdf_taz['SUBAREAID']==1]
gdf_taz_wf= gdf_taz_wf[['SA_TAZID','CO_TAZID']].sort_values(by='SA_TAZID').reset_index(drop=True)
gdf_taz_wf.rename(columns={'SA_TAZID':'TAZID'}, inplace=True)
gdf_taz_wf

,TAZID,CO_TAZID
0,1,30001
1,2,30002
2,3,30003
3,4,30004
4,5,30005
...,...,...
3541,3542,491326
3542,3543,491327
3543,3544,491328
3544,3545,491329


In [9]:
df_ltmile_cotaz = df_ltmile.merge(gdf_taz_wf, left_on='pTAZID', right_on='TAZID', how='left').rename(columns={'CO_TAZID':'pCO_TAZID'}).drop(columns=['TAZID'])
df_ltmile_cotaz = df_ltmile_cotaz.merge(gdf_taz_wf, left_on='aTAZID', right_on='TAZID', how='left').rename(columns={'CO_TAZID':'aCO_TAZID'}).drop(columns=['TAZID'])
df_ltmile_cotaz.dropna(inplace=True)
df_ltmile_cotaz

,pTAZID,aTAZID,pCO_TAZID,aCO_TAZID
0,1,1,30001,30001
1,1,2,30001,30002
3,2,1,30002,30001
4,2,2,30002,30002
6,3,3,30003,30003
...,...,...,...,...
21901,3513,3513,491297,491297
21902,3516,3516,491300,491300
21903,3520,3520,491304,491304
21904,3521,3521,491305,491305


In [10]:
import geopandas as gpd

schools_gdf = gpd.read_file(r'data\WFTDMv9.0-school-locations\Utah_Schools_PreK_to_12.shp')

# add column if Grade1 through Grade8 > 100 then 'K8' else if Grade9 through Grade12 > 100 then '1012' else 'other'
schools_gdf['HBSch_lev'] = schools_gdf.apply(lambda x: 'primary' if x[['Grade1', 'Grade2', 'Grade3', 'Grade4', 'Grade5', 'Grade6', 'Grade7', 'Grade8']].sum() > 100 else ('secondary' if x[['Grade10', 'Grade11', 'Grade12']].sum() > 100 else 'other'), axis=1)
schools_gdf = schools_gdf[schools_gdf['HBSch_lev'] != 'other']
schools_gdf


,OBJECTID,LEAName,SchoolName,LEANumber,LEAID,SchoolID,SchoolNumb,CharterSch,PrivateSch,OnlineScho,...,MultipleRa,PacificIsl,White,Economical,EnglishLea,StudentDis,Homeless,Preschool,geometry,HBSch_lev
0,1,Alpine District,River Rock School,1,122,186425,201.0,None,None,None,...,46.0,0.0,809.0,61.0,2.0,86.0,0.0,54.0,POINT (-111.87614 40.394),primary
1,2,Alpine District,Skyridge High School,1,122,186426,785.0,None,None,None,...,102.0,55.0,2517.0,433.0,43.0,185.0,3.0,0.0,POINT (-111.84812 40.42603),secondary
2,3,Alpine District,Springside School,1,122,186467,202.0,None,None,None,...,41.0,3.0,794.0,142.0,8.0,61.0,0.0,7.0,POINT (-111.91291 40.3493),primary
5,6,American Academy of Innovation,American Academy of Innovation,8K,186429,186430,700.0,Y,None,None,...,23.0,0.0,303.0,88.0,6.0,41.0,3.0,0.0,POINT (-112.02018 40.55691),primary
7,8,American Preparatory Academy,American Preparatory Academy - Accelerated School,74,110573,186243,110.0,Y,None,None,...,25.0,43.0,324.0,873.0,373.0,147.0,0.0,0.0,POINT (-111.978 40.70472),primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,1234,Utah Military Academy,Utah Military Academy - Camp Williams,2K,186369,186446,710.0,Y,None,None,...,26.0,9.0,326.0,124.0,28.0,88.0,0.0,0.0,POINT (-111.91113 40.41455),primary
1236,1237,Wasatch District,Daniels Canyon School,32,992,186521,116.0,None,None,None,...,5.0,2.0,387.0,223.0,110.0,52.0,9.0,63.0,POINT (-111.40353 40.47753),primary
1237,1238,Wasatch District,Timpanogos Middle School,32,992,186520,314.0,None,None,None,...,21.0,0.0,775.0,315.0,90.0,96.0,15.0,0.0,POINT (-111.39256 40.49682),primary
1239,1240,Weber District,Orchard Springs,35,1050,186569,142.0,None,None,None,...,7.0,2.0,324.0,147.0,27.0,36.0,1.0,17.0,POINT (-112.00046 41.31792),primary


In [11]:
hts_hbsch_trip_23 = client.query(
    "SELECT hh_id, pCO_TAZID_USTMv3, aCO_TAZID_USTMv3, PURP7_t, HBSch_lev, trip_weight_v2 "
    "FROM `confidential-2023-utah-hts.hts_2023_tdm_proc.trip` "
    "WHERE PURP7_t='HBSch' AND trip_weight_v2>0"
).to_dataframe()

hts_hh_loc_23 = client.query(
    "SELECT hh_id, home_lat, home_lon "
    "FROM confidential-2023-utah-hts.20250728.core_hh"
).to_dataframe()

hts_hbsch_trip_with_hh_loc_23 = pd.merge(
    hts_hbsch_trip_23, hts_hh_loc_23, on='hh_id', how='left'
)

hts_hbsch_trip_with_hh_loc_23.rename(columns={'trip_weight_v2': 'weight'}, inplace=True)
hts_hbsch_trip_with_hh_loc_23.drop(columns=['hh_id'], inplace=True)
hts_hbsch_trip_with_hh_loc_23['survey'] = 'HTS_2023'

hts_hbsch_trip_with_hh_loc_23


,pCO_TAZID_USTMv3,aCO_TAZID_USTMv3,PURP7_t,HBSch_lev,weight,home_lat,home_lon,survey
0,3059,3047,HBSch,primary,74.496660,41.78645,-112.10363,HTS_2023
1,3078,3047,HBSch,primary,201.674060,41.75757,-112.09391,HTS_2023
2,3087,3083,HBSch,primary,41.070107,41.73115,-112.16456,HTS_2023
3,3087,3117,HBSch,secondary,41.070107,41.73115,-112.16456,HTS_2023
4,3087,3083,HBSch,primary,41.070107,41.73115,-112.16456,HTS_2023
...,...,...,...,...,...,...,...,...
1642,490632,490668,HBSch,secondary,257.869800,40.26284,-111.70109,HTS_2023
1643,490597,490717,HBSch,secondary,223.865230,40.31391,-111.70647,HTS_2023
1644,490597,490717,HBSch,secondary,223.865230,40.31391,-111.70647,HTS_2023
1645,490632,490778,HBSch,secondary,38.721275,40.24462,-111.64107,HTS_2023


In [12]:
hts_hbsch_trip_with_hh_loc_12 = client.query(
    "SELECT p_CoTAZID_v30, a_CoTAZID_v30, PURP7_t, HBSch_lev, weight, hlat, hlon "
    "FROM `confidential-2023-utah-hts.previous_hts_2012_v30.trip` "
    "WHERE PURP7_t = 'HBSch' AND HBSch_lev != 'unidentified'"
).to_dataframe()

hts_hbsch_trip_with_hh_loc_12.rename(
    columns={
        'p_CoTAZID_v30': 'pCO_TAZID_USTMv3',
        'a_CoTAZID_v30': 'aCO_TAZID_USTMv3',
        'hlat': 'home_lat',
        'hlon': 'home_lon'
    },
    inplace=True
)
hts_hbsch_trip_with_hh_loc_12['survey'] = 'HTS_2012'

hts_hbsch_trip_with_hh_loc_12


,pCO_TAZID_USTMv3,aCO_TAZID_USTMv3,PURP7_t,HBSch_lev,weight,home_lat,home_lon,survey
0,30024,30025,HBSch,primary,57.900292,41.524491,-112.027137,HTS_2012
1,30024,30055,HBSch,secondary,57.900292,41.524491,-112.027137,HTS_2012
2,30024,30025,HBSch,primary,57.900292,41.524491,-112.027137,HTS_2012
3,30024,30025,HBSch,primary,57.900292,41.524491,-112.027137,HTS_2012
4,30030,30025,HBSch,primary,73.089105,41.518485,-112.020203,HTS_2012
...,...,...,...,...,...,...,...,...
3444,530251,530277,HBSch,primary,84.408552,37.071301,-113.590116,HTS_2012
3445,530251,530277,HBSch,primary,84.408552,37.071301,-113.590116,HTS_2012
3446,530251,530286,HBSch,primary,84.408552,37.071301,-113.590116,HTS_2012
3447,530278,530277,HBSch,primary,49.940499,37.056004,-113.568356,HTS_2012


In [13]:
hts_hbsch_trips = pd.concat(
    [hts_hbsch_trip_with_hh_loc_12, hts_hbsch_trip_with_hh_loc_23],
    ignore_index=True
)
hts_hbsch_trips = hts_hbsch_trips[hts_hbsch_trips['HBSch_lev']!='undefined']

hts_hbsch_trips


,pCO_TAZID_USTMv3,aCO_TAZID_USTMv3,PURP7_t,HBSch_lev,weight,home_lat,home_lon,survey
0,30024,30025,HBSch,primary,57.900292,41.524491,-112.027137,HTS_2012
1,30024,30055,HBSch,secondary,57.900292,41.524491,-112.027137,HTS_2012
2,30024,30025,HBSch,primary,57.900292,41.524491,-112.027137,HTS_2012
3,30024,30025,HBSch,primary,57.900292,41.524491,-112.027137,HTS_2012
4,30030,30025,HBSch,primary,73.089105,41.518485,-112.020203,HTS_2012
...,...,...,...,...,...,...,...,...
5091,490632,490668,HBSch,secondary,257.869800,40.262840,-111.701090,HTS_2023
5092,490597,490717,HBSch,secondary,223.865230,40.313910,-111.706470,HTS_2023
5093,490597,490717,HBSch,secondary,223.865230,40.313910,-111.706470,HTS_2023
5094,490632,490778,HBSch,secondary,38.721275,40.244620,-111.641070,HTS_2023


In [14]:
hts_hbsch_trips_agg = (
    hts_hbsch_trips
    .groupby(['survey', 'home_lat', 'home_lon', 'pCO_TAZID_USTMv3', 'aCO_TAZID_USTMv3', 'HBSch_lev'])
    .agg({'weight': 'sum'})
    .reset_index()
)

# if pCO_TAZID_USTMv3 = aCO_TAZID_USTMv3 add sameTaz field
hts_hbsch_trips_agg['sameTaz'] = hts_hbsch_trips_agg['pCO_TAZID_USTMv3'] == hts_hbsch_trips_agg['aCO_TAZID_USTMv3']

hts_hbsch_trips_agg = hts_hbsch_trips_agg.merge(df_ltmile_cotaz, left_on=['pCO_TAZID_USTMv3','aCO_TAZID_USTMv3'], right_on=['pCO_TAZID','aCO_TAZID'], how='left')

# Add boolean field: True if join matched, False otherwise
hts_hbsch_trips_agg['lt1mile'] = ~hts_hbsch_trips_agg['pCO_TAZID'].isna()


hts_hbsch_trips_agg


,survey,home_lat,home_lon,pCO_TAZID_USTMv3,aCO_TAZID_USTMv3,HBSch_lev,weight,sameTaz,pTAZID,aTAZID,pCO_TAZID,aCO_TAZID,lt1mile
0,HTS_2012,37.042957,-112.516115,25052,25046,primary,55.257359,False,NaN,NaN,<NA>,<NA>,False
1,HTS_2012,37.042957,-112.516115,25052,25056,secondary,221.029437,False,NaN,NaN,<NA>,<NA>,False
2,HTS_2012,37.042957,-112.516115,25052,25057,secondary,110.514719,False,NaN,NaN,<NA>,<NA>,False
3,HTS_2012,37.046845,-113.607092,530285,530277,primary,286.849422,False,NaN,NaN,<NA>,<NA>,False
4,HTS_2012,37.046845,-113.607092,530285,530285,secondary,71.712355,True,NaN,NaN,<NA>,<NA>,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2223,HTS_2023,41.860720,-111.987390,50099,50225,secondary,692.622135,False,NaN,NaN,<NA>,<NA>,False
2224,HTS_2023,41.861320,-111.999330,50098,50548,primary,19.505102,False,NaN,NaN,<NA>,<NA>,False
2225,HTS_2023,41.868990,-111.985400,50697,50225,secondary,42.983887,False,NaN,NaN,<NA>,<NA>,False
2226,HTS_2023,41.889140,-111.897910,50075,50145,primary,24.966762,False,NaN,NaN,<NA>,<NA>,False


In [15]:
import folium
import pandas as pd
from ipywidgets import HBox, VBox, Label, SelectMultiple, Output
from IPython.display import display, clear_output

# --- Widgets: options pulled from your DataFrame ---
survey_opts = tuple(sorted(hts_hbsch_trips_agg['survey'].dropna().unique()))
schlev_opts   = tuple(sorted(hts_hbsch_trips_agg['HBSch_lev'].dropna().unique()))

w_survey = SelectMultiple(
    options=survey_opts,
    value=survey_opts,               # default: all selected
    rows=min(len(survey_opts), 8),
    description='Survey',
    layout={'width': '300px'}
)

w_schlev = SelectMultiple(
    options=schlev_opts,
    value=schlev_opts,                 # default: all selected
    rows=min(len(schlev_opts), 8),
    description='HBSch_lev',
    layout={'width': '300px'}
)

out = Output()

def make_map(df):
    """Return a Folium map for the given (filtered) dataframe."""
    if df.empty:
        return None

    # Center on filtered data
    center_lat = float(df['home_lat'].mean())
    center_lon = float(df['home_lon'].mean())
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles="cartodbpositron")

    # Scale dot size by max weight in the filtered set
    max_wt = float(df['weight'].max()) if pd.notnull(df['weight'].max()) else 0.0
    scale = 60.0  # tweak this if you want larger/smaller max circle
    denom = max_wt if max_wt > 0 else 1.0

    for _, row in df.iterrows():
        r = (row['weight'] / denom) * scale
        folium.CircleMarker(
            location=[row['home_lat'], row['home_lon']],
            radius=float(r),
            color="blue",
            fill=True,
            fill_color="blue",
            fill_opacity=0.6,
            popup=folium.Popup(
                f"<b>survey:</b> {row.get('survey','')}"
                f"<br><b>HBSch_lev:</b> {row.get('HBSch_lev','')}"
                f"<br><b>weight:</b> {row.get('weight','')}", max_width=250
            ),
        ).add_to(m)

    return m

def update_map(*_):
    with out:
        clear_output(wait=True)
        # If either widget has nothing selected, show a helpful message
        if len(w_survey.value) == 0 or len(w_schlev.value) == 0:
            display(Label("Select at least one value in both filters to draw the map."))
            return

        df_f = hts_hbsch_trips_agg[
            hts_hbsch_trips_agg['survey'].isin(w_survey.value) &
            hts_hbsch_trips_agg['HBSch_lev'].isin(w_schlev.value)
        ].copy()

        m = make_map(df_f)
        if m is None:
            display(Label("No rows after filtering."))
        else:
            display(m)

# Wire up reactive updates
w_survey.observe(update_map, names='value')
w_schlev.observe(update_map, names='value')

# Render UI + initial map
#controls = HBox([w_survey, w_schlev])
#display(VBox([controls, out]))
#update_map()


In [16]:
import folium
from folium.plugins import DualMap
import pandas as pd
from ipywidgets import SelectMultiple, VBox, HTML, Output
from IPython.display import display, clear_output

df = hts_hbsch_trips_agg.copy()

# --- Surveys for left/right maps ---
left_survey  = 'HTS_2012'
right_survey = 'HTS_2023'

# --- HBSch_lev filter widget ---
schlev_opts = tuple(sorted(df['HBSch_lev'].dropna().unique()))
w_schlev = SelectMultiple(
    options=schlev_opts,
    value=schlev_opts,   # default: all selected
    rows=min(len(schlev_opts), 8),
    description='HBSch_lev',
    layout={'width': '300px'}
)

out = Output()
title = HTML(
    f"<h3 style='text-align:center;'>Comparison of {left_survey} (left) vs {right_survey} (right)</h3>"
)

# --- Helpers ---
def bounds_for(df_):
    return [[df_['home_lat'].min(), df_['home_lon'].min()],
            [df_['home_lat'].max(), df_['home_lon'].max()]]

def add_circles(map_obj, df_side, color='blue', scale=40.0):
    if df_side.empty:
        return
    max_wt = float(df_side['weight'].max()) if pd.notnull(df_side['weight'].max()) else 0.0
    denom = max_wt if max_wt > 0 else 1.0
    for _, row in df_side.iterrows():
        r = float((row['weight'] / denom) * scale)
        folium.CircleMarker(
            location=[row['home_lat'], row['home_lon']],
            radius=r,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=folium.Popup(
                f"<b>survey:</b> {row.get('survey','')}"
                f"<br><b>HBSch_lev:</b> {row.get('HBSch_lev','')}"
                f"<br><b>weight:</b> {row.get('weight','')}", max_width=250
            ),
        ).add_to(map_obj)

def make_dualmap(df_f):
    if df_f.empty:
        return None

    center = [df_f['home_lat'].mean(), df_f['home_lon'].mean()]
    dual = DualMap(location=center, zoom_start=10, tiles='cartodbpositron')

    # Split into surveys
    df_left  = df_f[df_f['survey'] == left_survey]
    df_right = df_f[df_f['survey'] == right_survey]

    add_circles(dual.m1, df_left,  color='blue')
    add_circles(dual.m2, df_right, color='red')

    # Fit both to overall bounds
    b = bounds_for(df)
    dual.m1.fit_bounds(b)
    dual.m2.fit_bounds(b)

    return dual

def update_map(*_):
    with out:
        clear_output(wait=True)
        df_f = df[df['HBSch_lev'].isin(w_schlev.value)].copy()
        m = make_dualmap(df_f)
        display(title, m if m else "No data for selection.")

# Wire up
w_schlev.observe(update_map, names='value')

# UI
#display(VBox([w_schlev, out]))
#update_map()


In [17]:
import folium
from folium.plugins import DualMap
import pandas as pd
from ipywidgets import SelectMultiple, VBox, HTML, Output, Label
from IPython.display import display, clear_output

df = hts_hbsch_trips_agg.copy()

# --- Ensure schools are WGS84 (lat/lon) ---
schools = schools_gdf.copy()
if getattr(schools, "crs", None) is None or schools.crs.to_string().upper() != "EPSG:4326":
    schools = schools.to_crs(epsg=4326)

# --- Load TAZ polygons ---
taz = gpd.read_file(r"data\TAZ\WFv910_TAZ.shp")
if getattr(taz, "crs", None) is None or taz.crs.to_string().upper() != "EPSG:4326":
    taz = taz.to_crs(epsg=4326)

# --- Surveys for left/right maps ---
left_survey  = 'HTS_2012'
right_survey = 'HTS_2023'

# --- HBSch_lev filter widget ---
lev_df = set(df['HBSch_lev'].dropna().unique()) if 'HBSch_lev' in df.columns else set()
lev_sch = set(schools['HBSch_lev'].dropna().unique()) if 'HBSch_lev' in schools.columns else set()
schlev_opts = tuple(sorted(lev_df.union(lev_sch)))
w_schlev = SelectMultiple(
    options=schlev_opts,
    value=schlev_opts,
    rows=min(len(schlev_opts), 8) if schlev_opts else 1,
    description='HBSch_lev',
    layout={'width': '300px'}
)

out = Output()
title = HTML(
    f"<h3 style='text-align:center;'>Comparison of {left_survey} (left) vs {right_survey} (right)</h3><h4 style='text-align:center;'>Red: PA < 1 Mile --- Blue: PA >=1 Mile</h4>"
)

# --- Helpers ---
def bounds_for(df_):
    return [[df_['home_lat'].min(), df_['home_lon'].min()],
            [df_['home_lat'].max(), df_['home_lon'].max()]]

def add_circles(map_obj, df_side, scale=40.0):
    if df_side.empty:
        return

    # normalize sizes within this subset
    max_wt = float(df_side['weight'].max()) if pd.notnull(df_side['weight'].max()) else 0.0
    denom = max_wt if max_wt > 0 else 1.0

    for _, row in df_side.iterrows():
        r = float((row['weight'] / denom) * scale)

        # Treat lt1mile truthy if it's True/1/"true"/"yes"
        v = str(row.get('lt1mile', '')).strip().lower()
        is_lt1mile = bool(row.get('lt1mile')) or v in ('true', '1', 'yes', 'y')

        edge_color = 'red' if is_lt1mile else 'blue'
        edge_weight = 1 #2.5 if is_lt1mile else 1.0

        folium.CircleMarker(
            location=[row['home_lat'], row['home_lon']],
            radius=r,
            color=edge_color,            # outline color
            weight=edge_weight,          # outline thickness
            fill=True,
            fill_color=edge_color,       # fill stays series color
            fill_opacity=0.6,
            popup=folium.Popup(
                f"<b>survey:</b> {row.get('survey','')}"
                f"<br><b>HBSch_lev:</b> {row.get('HBSch_lev','')}"
                f"<br><b>weight:</b> {row.get('weight','')}"
                f"<br><b>lt1mile:</b> {row.get('lt1mile','')}", 
                max_width=250
            ),
        ).add_to(map_obj)

def add_schools_layer(map_obj, schools_gdf, layer_name="Schools"):
    """Add schools as square markers (green for primary, blue for secondary)."""
    if schools_gdf.empty:
        return
    fg = folium.FeatureGroup(name=layer_name, show=True)
    for _, srow in schools_gdf.iterrows():
        geom = srow.geometry
        if geom is None:
            continue
        # If not a Point, use centroid
        lat, lon = (geom.y, geom.x) if geom.geom_type == "Point" else (geom.centroid.y, geom.centroid.x)
        
        name = srow.get('name') or srow.get('SCHOOL') or srow.get('SchoolName') or 'School'
        folium.RegularPolygonMarker(
            location=[lat, lon],
            number_of_sides=4,   # square
            radius=4,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.9,
            tooltip=name,
            popup=folium.Popup(
                f"<b>School:</b> {name}"
                f"<br><b>HBSch_lev:</b> {srow.get('HBSch_lev','')}", max_width=250
            )
        ).add_to(fg)
    fg.add_to(map_obj)

def add_taz_layer(map_obj, taz_gdf, layer_name="TAZ Boundaries"):
    """Add polygon outlines with no fill."""
    if taz_gdf.empty:
        return
    style = lambda x: {"fillOpacity": 0, "weight": 2, "color": "green"}
    folium.GeoJson(
        taz_gdf,
        name=layer_name,
        style_function=style
    ).add_to(map_obj)

def make_dualmap(df_f, schools_f, taz_gdf):
    if df_f.empty and schools_f.empty and taz_gdf.empty:
        return None

    center = [df['home_lat'].mean(), df['home_lon'].mean()]
    dual = DualMap(location=center, zoom_start=10, tiles='CartoDB positron nolabels')

    # Split trips by survey
    df_left  = df_f[df_f['survey'] == left_survey]
    df_right = df_f[df_f['survey'] == right_survey]

    add_taz_layer(dual.m1, taz_gdf, layer_name="TAZ Boundaries")
    add_taz_layer(dual.m2, taz_gdf, layer_name="TAZ Boundaries")

    # Schools
    add_schools_layer(dual.m1, schools_f, layer_name="Schools")
    add_schools_layer(dual.m2, schools_f, layer_name="Schools")

    # Trips
    add_circles(dual.m1, df_left)
    add_circles(dual.m2, df_right)

    # Fit to trips extent
    if not df.empty:
        b = bounds_for(df)
        dual.m1.fit_bounds(b)
        dual.m2.fit_bounds(b)

    folium.LayerControl(collapsed=False).add_to(dual.m1)
    folium.LayerControl(collapsed=False).add_to(dual.m2)

    return dual

def update_map(*_):
    with out:
        clear_output(wait=True)
        if len(w_schlev.value) == 0:
            display(Label("Select at least one HBSch_lev to draw the maps."))
            return

        # Filter
        df_f = df[df['HBSch_lev'].isin(w_schlev.value)].copy()
        schools_f = schools[schools['HBSch_lev'].isin(w_schlev.value)].copy()

        m = make_dualmap(df_f, schools_f, taz)
        display(title, m if m else "No data for selection.")

# Wire up
w_schlev.observe(update_map, names='value')

# UI
display(VBox([w_schlev, out]))
update_map()
